In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from time import time
from torch import nn, optim
import torch.nn.functional as F

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

In [3]:
train_set = datasets.MNIST('./data/train/', 
                           download  = True, 
                           train     = True, 
                           transform = transform)

 99%|███████████████████████████████████████████████████████████████████▌| 9854976/9912422 [00:45<00:00, 392689.72it/s]

Extracting ./data/train/MNIST\raw\train-images-idx3-ubyte.gz



0it [00:00, ?it/s]
 57%|█████████████████████████████████████████▍                               | 16384/28881 [00:01<00:00, 45596.22it/s]
32768it [00:01, 29539.54it/s]                                                                                          

Extracting ./data/train/MNIST\raw\train-labels-idx1-ubyte.gz



0it [00:00, ?it/s]
 99%|███████████████████████████████████████████████████████████████████▌| 1638400/1648877 [00:10<00:00, 216763.23it/s]

Extracting ./data/train/MNIST\raw\t10k-images-idx3-ubyte.gz




0it [00:00, ?it/s]

  0%|                                                                                         | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 14564.96it/s]                                                                                           

Extracting ./data/train/MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
test_set = datasets.MNIST('./data/test/', 
                          download  = True, 
                          train     = False, 
                          transform = transform)



9920512it [01:00, 392689.72it/s]                                                                                       

  0%|                                                                                      | 0/9912422 [00:01<?, ?it/s]

  0%|                                                                       | 16384/9912422 [00:01<02:02, 80669.92it/s]

  0%|▏                                                                      | 24576/9912422 [00:02<03:01, 54335.72it/s]

  0%|▏                                                                      | 32768/9912422 [00:02<03:26, 47862.60it/s]

  0%|▎                                                                      | 40960/9912422 [00:02<03:49, 43087.95it/s]

  0%|▎                                                                      | 49152/9912422 [00:02<03:19, 49380.01it/s]

  1%|▍                                                                      | 57344/9912422 [00:02<03:33, 46266.82it/s]

  1%|▍                        

Extracting ./data/test/MNIST\raw\train-images-idx3-ubyte.gz





0it [00:00, ?it/s]


  0%|                                                                                        | 0/28881 [00:00<?, ?it/s]


 57%|█████████████████████████████████████████▍                               | 16384/28881 [00:01<00:00, 34955.36it/s]


32768it [00:01, 23833.02it/s]                                                                                          

Extracting ./data/test/MNIST\raw\train-labels-idx1-ubyte.gz





0it [00:00, ?it/s]


  0%|                                                                                      | 0/1648877 [00:01<?, ?it/s]


  1%|▋                                                                      | 16384/1648877 [00:01<01:19, 20562.29it/s]


  2%|█▍                                                                     | 32768/1648877 [00:02<01:19, 20441.91it/s]


  2%|█▊                                                                     | 40960/1648877 [00:04<02:10, 12321.25it/s]


  3%|██                                                                     | 49152/1648877 [00:04<01:59, 13413.91it/s]


  4%|██▊                                                                    | 65536/1648877 [00:05<01:42, 15438.72it/s]


  5%|███▌                                                                   | 81920/1648877 [00:05<01:32, 17010.67it/s]


  6%|████▏                                                                  | 98304/1648877 [00:06<01:28, 17491.30it/s]




Extracting ./data/test/MNIST\raw\t10k-images-idx3-ubyte.gz






0it [00:00, ?it/s]



  0%|                                                                                         | 0/4542 [00:00<?, ?it/s]



8192it [00:00, 16913.80it/s]                                                                                           

Extracting ./data/test/MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [8]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(400, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 400)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [39]:
device = torch.device("cpu")
model = Net()

In [28]:
model = nn.Sequential(nn.Conv2d(1, 8, 3),
                      nn.ReLU(),
                      nn.Conv2d(8, 16, 3),
                      nn.ReLU(),
                      nn.Linear(16*8*8, 100),
                      nn.ReLU(),
                      nn.Linear(100, 10),
                      nn.LogSoftmax(dim = 1))

In [30]:
loss_function = nn.NLLLoss()

In [40]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [42]:
time0 = time()
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))

Epoch 0 - Training loss: 0.4219799446406713
Epoch 1 - Training loss: 0.08289434520134539
Epoch 2 - Training loss: 0.05862108637481483
Epoch 3 - Training loss: 0.046089242206517056
Epoch 4 - Training loss: 0.03800240151885984
Epoch 5 - Training loss: 0.03235981112588316
Epoch 6 - Training loss: 0.028607837879645038
Epoch 7 - Training loss: 0.02484251156465204
Epoch 8 - Training loss: 0.021964005490482996
Epoch 9 - Training loss: 0.019312029347014325
Epoch 10 - Training loss: 0.016767851805794976
Epoch 11 - Training loss: 0.015464184059104177
Epoch 12 - Training loss: 0.013169094427689307
Epoch 13 - Training loss: 0.011738310292013672
Epoch 14 - Training loss: 0.010845196685557171


In [46]:
correct_count, all_count = 0, 0
for images, labels in test_loader:
    for i in range(len(labels)):
        img = images[i].view(1, 1, 28, 28)
        with torch.no_grad():
            logps = model(img)

    
        ps = torch.exp(logps)
        probab = list(ps.numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
            correct_count += 1
        all_count += 1

print("Number Of Images Tested :", all_count)
print("\nModel Accuracy :", (correct_count/all_count))

Number Of Images Tested : 10000

Model Accuracy : 0.991


In [48]:
torch.save(model, './mnist_model.pt') 